## Requests в Python

Библиотека requests является стандартом де-факто для реализации отправки HTTP-запросов в Python. 

В целом процесс: клиент (например браузер или скрипт Python, использующий библиотеку requests) отправляет данные на URL, а сервер с этим URL считывает данные, решает, что с ними делать, и отправляет клиенту ответ. После этого клиент может решить, что делать с полученными в ответе данными.
В составе запроса клиент отправляет данные по методу запроса. Наиболее распространенными методами запроса являются GET, POST и PUT. Запросы GET обычно предназначены только для чтения данных без их изменения, а запросы POST и PUT обычно предназначаются для изменения данных на сервере.


## Метод GET

Метод GET предписывает, что вы пытаетесь получить или извлечь некоторые данные из указанного ресурса. Чтобы отправить GET запрос, необходимо вызвать метод requests.get(url).

In [4]:
import requests

response = requests.get('http://httpbin.org')

Такой же запрос делает и браузер для просмотра этой страницы. Отличие в том, что библиотека requests не может выполнить рендеринг кода HTML, поэтому мы получим просто код HTML и другую информацию, содержащуюся в ответе.

## Коды состояния

Коты о кодах: <https://http.cat>

* 1XX — информационный сообщения, например: 102 Processing — запрос принят, но на его обработку понадобится длительное время. 
* 2XX — информируют о случаях успешного принятия и обработки запроса клиента
* 3XX — сообщают клиенту, что для успешного выполнения операции необходимо сделать другой запрос, как правило, по другому URI
* 4XX — ошибка на стороне клиента
* 5XX — ошибка на стороне сервера

Обычно при выполнении запросов мы хотим получить коды состояния в диапазоне 200.

Библиотека requests понимает, что коды состояния 4XX и 5XX сигнализируют об ошибках, и поэтому при возврате этих кодов состояния объекту ответа на запрос присваивается значение False. Также вместо условия if можно генерировать исключение специального типа HTTPError, если запрос был неудачен. Сделать это можно используя метод **.raise_for_status()**

In [5]:
if response:
    print ('OK')
else:
    print('Wrong')

OK


Посмотреть статус код:

In [6]:
print(response.status_code)  # >>> 200

200


## headers

Заголовки ответа сервера могут дать много полезной информации, например, тип содержимого ответа, ограничение по времени, в течение которого ответ будет кэшироваться и т.д. Чтобы просмотреть содержимое заголовков, необходимо обратиться к свойству объекта Response.headers

Заголовки отправляются вместе с запросом и возвращаются с ответом. Заголовки используются для того, чтобы клиент и сервер понимали, как интерпретировать данные, отправляемые и получаемые в запросе и ответе.

Заголовок **content type** показывает формат данных, например HTML, JSON, PDF, обычный текст и т.д.
При обращении к свойству Response.headers будет возвращен схожий со словарем объект, позволяющий получить доступ к значениям заголовков полученного ответа по ключу. Например, чтобы определить тип полезного содержимого ответа, получаем доступ к значению заголовка Content-Type:

*Спецификация HTTP определяет названия заголовков без учета регистра! Можно обращаться к заголовкам без учета регистра.*

Посмотреть заголовки (словарь):

In [20]:
print(response.headers)
print(response.headers['Content-Type'])

{'Date': 'Fri, 25 Mar 2022 10:22:27 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '9593', 'Connection': 'keep-alive', 'Server': 'gunicorn/19.9.0', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true'}
text/html; charset=utf-8


## .text & .content & .json() 

Ответ на успешный запрос GET часто содержит в сообщении полезное содержимое (payload). Используя атрибуты и методы объекта Response, мы можем просматривать его в различных форматах.

* Атрибут .text возвращает cодержание/контент ответа сервера в юникоде, например для html-страницы, можно увидеть код html.
* Атрибут .content возвращает содержание ответа сервера, представленное в байтах (бинарный вид).

In [ ]:
print(response.text)
print(response.content)

Содержимое ответа также может представлять собой сериализованную строку в формате JSON. Поэтому для того, чтобы в результате получить словарь, можно взять строку str, полученную из свойства Response.text, и десериализовать ее с помощью метода json.loads(). Однако более простой способ выполнить эту задачу — использовать метод нашего объекта Response.json()

In [ ]:
response.json()

## Query String Parameters - параметры строки запроса

GET запрос не имеет тела сообщения. Но, это не означает, что с его помощью нельзя передать серверу никакую информацию. Это можно делать с помощью GET параметров. Чтобы добавить GET параметры к запросу, нужно в конце URL-адреса поставить знак ? и задавать их после него по следующему правилу: 

**имя_параметра1=значение_параметра1&имя_параметра2=значение_параметра2.** Разделителем между параметрами служит знак &.

Для передачи GET параметров запроса необходимо передать нужную информацию в именованный параметр params метода .get()

In [ ]:
import requests
# Поиск requests в репозиториях GitHub
response = requests.get(
    'https://api.github.com/search/repositories',
    params={'q': 'requests+language:python'},
)
# Просматриваем значения атрибутов результатов поиска репозитория requests
json_response = response.json()
repository = json_response['items'][0]
print(f'Repository name: {repository["name"]}')  # Python 3.6+
print(f'Repository description: {repository["description"]}')  # Python 3.6+


Можно передавать значения в params метода get() не только в виде словаря, но и в виде списка кортежей или бинарном виде:

In [22]:
params=[('q', 'requests+language:python'), ('p', 'some value')]
params=b'q=requests+language:python'